In [1]:
import pandas as pd

import datetime as dt

from datetime import date
from datetime import datetime


from datetime import timedelta



source = pd.read_csv('Merge.csv')

source['Datetime'] = pd.to_datetime(source['Datetime'])
source.sort_values(by='Datetime',ascending=False)
print(source)


#source['Date'] = source.Datetime.date()
arr_date = source['Datetime'].dt.to_pydatetime()
source['Datetime'] = pd.Series(arr_date, dtype=object)

#source['Datetime'] = pd.to_datetime(source['Datetime'])



#check for nulls in DateTime
#source['Datetime'].isnull().values.any()

source['Date'] = source['Datetime'].apply(lambda x: x.date())

#source['Date'].unique().tolist()[0]


#source['Resolution'].unique()

#check if need to order

list_of_times = []

for x, date_raw in enumerate(source['Date'].unique().tolist()):
    date = datetime.combine(date_raw,datetime.min.time())


    for y, number in enumerate(range(6)):
        list_of_times.append(date.replace(hour=7+2*(y)))

list_of_times

final_df = pd.DataFrame(index=list_of_times)


def time_modifier(time,spec):
    list_of_times = []
    if spec == '10MIN':
        for y, number in enumerate(range(12)):

            list_of_times.append(time+timedelta(minutes = 10*y))

    elif spec == '1H':
        for y, number in enumerate(range(2)):

            list_of_times.append(time+timedelta(hours = 1*y))
    
    elif spec == 'D':
        list_of_times.append(time.replace(hour=0))

    return list_of_times


def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))


def get_avg_and_f_fill(time,spec):


    list_of_times = time_modifier(time,spec)


    #---
    universe = source[source['Resolution'] == spec]

    #here we remove na values for a simple approach to calculate averages
    final_universe = universe[universe['Datetime'].isin(list_of_times)].dropna()
    #here we have a len check if == 0 then it will get the last known observation for ff-filling the D data

    
    if spec == 'D' and len(final_universe) == 0:
  
        search = universe.dropna()

        near = search['Datetime'][search['Datetime'] <=time].iloc[-1]

        #near = nearest(search['Datetime'], time)

        final_universe = universe[universe['Datetime'] == near]


    avg = final_universe.Price.mean()

    return avg


final_df['avg_10MIN'] = final_df.index.to_series().apply(lambda x: get_avg_and_f_fill(x,'10MIN'))

final_df['avg_1H'] = final_df.index.to_series().apply(lambda x: get_avg_and_f_fill(x,'1H'))

final_df['f_fill_D'] = final_df.index.to_series().apply(lambda x: get_avg_and_f_fill(x,'D'))

final_df = final_df.sort_index()
#final_df.to_csv('q1.csv')

     Resolution            Datetime   Price
0         10MIN 2021-11-01 07:00:00  70.000
1         10MIN 2021-11-01 07:10:00     NaN
2         10MIN 2021-11-01 07:20:00  71.500
3         10MIN 2021-11-01 07:30:00  71.375
4         10MIN 2021-11-01 07:40:00     NaN
...         ...                 ...     ...
1565          D 2021-11-26 00:00:00  88.350
1566          D 2021-11-27 00:00:00     NaN
1567          D 2021-11-28 00:00:00     NaN
1568          D 2021-11-29 00:00:00  94.900
1569          D 2021-11-30 00:00:00  91.700

[1570 rows x 3 columns]
